# **Preprocess the Data** 



Creating the fumctions for all different preprocess processess so we can handle all possibilities

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from collections import Counter
import os
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler



### **Loading the data**

lists of all possibilities

In [2]:
#listing the used possiblities

null_threshold_list = [10,20]
null_method_list = ["mean","median"]

In [3]:
def load_data(threshold,method):
    
    # this will help later for handeling each case
    
    folder_path = rf"handeled_null_values\{threshold}_threshold\using_{method}"

    train_data =  pd.read_csv(rf"{folder_path}\train_data_cleaned.csv")
    test_data =  pd.read_csv(rf"{folder_path}\test_data_cleaned.csv")
    
    print("\n**************************************\n")
    
    print(f"Handeling:  {folder_path}")
    
    return train_data, test_data
   

### **Preprocessing steps needed**

this is to check which other preprocessing steps are needed 

In [4]:
example_train_data = pd.read_csv(r"handeled_null_values\20_threshold\using_mean\train_data_cleaned.csv")
example_test_data = pd.read_csv(r"handeled_null_values\20_threshold\using_mean\test_data_cleaned.csv")


In [5]:
example_train_data.head()

Column0  Column1  Column2   Column3   Column4   Column6   Column7  \
0      2.0     2495   3726.0  0.678139  0.701403  0.434190 -0.015603   
1      0.0     2495   3454.0  0.452580  0.701403  1.554998 -0.015574   
2      2.0     2495   4543.0 -1.577453 -1.429540 -0.407939 -0.015607   
3      0.0      211     59.0 -0.000210 -0.000855 -0.407939 -0.015607   
4      0.0      718    950.0 -2.028572 -1.855728 -0.407939 -0.015607   

    Column8  Column10  Column11  Column12  Column13  Column15  Column16  \
0  0.606265         0         0         0         0   0.00339       0.0   
1  0.329946         0         0         0         0   0.00339       0.0   
2 -0.774979         1         1         1         1   0.00339       0.0   
3 -0.774979         0         0         0         0   0.00339       0.0   
4 -0.774979         0         0         0         0   0.00339       0.0   

   Column17  Column18  Column19  Column20  Column21  target  
0         0       0.0         0         0         0       0  
1         0       0.0         0         0         0       0  
2         0       0.0         0         0         0       0  
3         0       1.0         0         0         0       1  
4         0       0.0         0         0         0       0

In [6]:
def check_preprocessing_needs(df, target_column=None):
    
    
    report = {}

    # 1. Check for missing values # we have handled it already but for future purposes
    missing_values = df.isnull().sum().sum()
    if missing_values > 0:
        report['missing_values'] = f"Dataset contains {missing_values} missing values."
    else:
        report['missing_values'] = "No missing values detected."

    # 2. Check for duplicate rows
    duplicate_rows = df.duplicated().sum()
    if duplicate_rows > 0:
        report['duplicates'] = f"Dataset contains {duplicate_rows} duplicate rows."
    else:
        report['duplicates'] = "No duplicate rows detected."

    # 3. Check for outliers using Z-score method
    z_scores = np.abs(stats.zscore(df.select_dtypes(include=[np.number])))  # Only check numerical columns
    outliers = (z_scores > 4).sum().sum()
    if outliers > 0:
        report['outliers'] = f"Dataset contains {outliers} outliers based on Z-scores > 4."
    else:
        report['outliers'] = "No significant outliers detected."

    # 4. Check for need of scaling (Check if data is already scaled or normalized)
    max_range = df.select_dtypes(include=[np.number]).max() - df.select_dtypes(include=[np.number]).min()
    if any(max_range > 1):
        report['scaling'] = "Scaling is recommended (range > 1 found in some columns)."
    else:
        report['scaling'] = "Data seems to be scaled already (range <= 1)."

    # 5. Check for negative values in numerical columns (this is for later use so we can know which normalization steps to use)
    negative_columns = df.select_dtypes(include=[np.number]).columns[df.lt(0).any()]
    if len(negative_columns) > 0:
        report['negative_values'] = f"Columns with negative values: {list(negative_columns)}"
    else:
        report['negative_values'] = "No columns with negative values detected."

    # 6. Check for class imbalance if target_column is provided
    if target_column:
        class_distribution = Counter(df[target_column])
        majority_class = max(class_distribution.values())
        minority_class = min(class_distribution.values())
        imbalance_ratio = majority_class / minority_class

        # Threshold for imbalance: If majority class is 1.5x larger than minority, report imbalance
        if imbalance_ratio > 1.5:
            report['class_imbalance'] = f"Dataset is imbalanced. Majority class is {imbalance_ratio:.2f}x larger than minority class."
            report['class_distribution'] = f"Class distribution: {dict(class_distribution)}"
        else:
            report['class_imbalance'] = "No significant class imbalance detected."
            report['class_distribution'] = f"Class distribution: {dict(class_distribution)}"

    # 7. Summary of dataset shape and basic stats 
    report['data_shape'] = f"Dataset shape: {df.shape[0]} rows and {df.shape[1]} columns."
    
    return report

In [7]:


# Check if preprocessing is needed for the training data, 
train_report = check_preprocessing_needs(example_train_data, target_column='target')
test_report = check_preprocessing_needs(example_test_data, target_column='target')

print("Training Data Report:")
for key, value in train_report.items():
    print(f"{key}: {value}")

print("\nTest Data Report:")
for key, value in test_report.items():
    print(f"{key}: {value}")

Training Data Report:
missing_values: No missing values detected.
duplicates: Dataset contains 14617 duplicate rows.
outliers: Dataset contains 51814 outliers based on Z-scores > 4.
scaling: Scaling is recommended (range > 1 found in some columns).
negative_values: Columns with negative values: ['Column1', 'Column2', 'Column3', 'Column4', 'Column6', 'Column7', 'Column8', 'Column15']
class_imbalance: Dataset is imbalanced. Majority class is 9.61x larger than minority class.
class_distribution: Class distribution: {0: 711100, 1: 74033}
data_shape: Dataset shape: 785133 rows and 20 columns.

Test Data Report:
missing_values: No missing values detected.
duplicates: Dataset contains 2443 duplicate rows.
outliers: Dataset contains 17214 outliers based on Z-scores > 4.
scaling: Scaling is recommended (range > 1 found in some columns).
negative_values: Columns with negative values: ['Column1', 'Column2', 'Column3', 'Column4', 'Column6', 'Column7', 'Column8', 'Column15']
class_imbalance: Datase

### **Preprocessing the data**

we are doing all the preprocessing steps before scaling the data

In [8]:
def preprocess_data(
    null_method, 
    null_threshold, 
    imbalance_method,
    dim_reduction_method,
    corr_threshold ,
    pca_components,
    z_threshold,
    total_folder_list
):
   
    folder_path = rf"handeled_null_values\{null_threshold}_threshold\using_{null_method}"
    
    train_data =  pd.read_csv(rf"{folder_path}\train_data_cleaned.csv")
    test_data=  pd.read_csv(rf"{folder_path}\test_data_cleaned.csv")
    #drop duplicates
    train_data = train_data.drop_duplicates().reset_index(drop=True)
    test_data = test_data.drop_duplicates().reset_index(drop=True)
    
    numerical_cols = train_data.columns.difference(['target'])
    z_scores = np.abs(stats.zscore(train_data[numerical_cols]))

    # Create a boolean mask where all Z-scores are below the threshold
    outlier_mask = (z_scores < z_threshold).all(axis=1)

    # Apply the mask to the entire dataset and reset the index
    train_data_clean = train_data[outlier_mask].reset_index(drop=True)

    # Now separate features and target
    train_target_clean = train_data_clean['target']
    train_data_clean = train_data_clean.drop(columns=['target'])
    test_target = test_data['target']
    test_data = test_data.drop(columns=['target'])
    
    
    # Handle Class Imbalance #
    # Calculate class distribution
    class_counts = train_target_clean.value_counts()
    imbalance_ratio = class_counts.max() / class_counts.min()
    print(f"Imbalance Ratio: {imbalance_ratio}")

    if imbalance_method == 'undersampling':
        # Apply random undersampling
        rus = RandomUnderSampler(
            sampling_strategy=0.5, 
            random_state=42
        )
        train_data_balanced, train_target_balanced = rus.fit_resample(train_data_clean, train_target_clean)

    elif imbalance_method == 'smote':
        # Apply SMOTE (oversampling)
        smote = SMOTE(
            sampling_strategy='auto', 
            random_state=42
        )
        train_data_balanced, train_target_balanced = smote.fit_resample(train_data_clean, train_target_clean)

    elif imbalance_method == 'both':
        # Adjust sampling strategies
        # First, undersample the majority class
        rus = RandomUnderSampler(
            sampling_strategy=0.5,  # Reduce majority class to be twice the minority class
            random_state=42
        )
        train_data_under, train_target_under = rus.fit_resample(train_data_clean, train_target_clean)

        # Then, oversample the minority class to balance the dataset
        smote = SMOTE(
            sampling_strategy=1.0,  # Balance the classes equally
            random_state=42
        )
        train_data_balanced, train_target_balanced = smote.fit_resample(train_data_under, train_target_under)
    else:
        # No imbalance handling
        train_data_balanced = train_data_clean.copy()
        train_target_balanced = train_target_clean.copy()

    # Dimensionality Reduction #
    if dim_reduction_method == 'pca':
        # Determine optimal number of components if not specified
        if pca_components is None:
            pca_temp = PCA().fit(train_data_balanced)
            cumulative_variance = np.cumsum(pca_temp.explained_variance_ratio_)
            pca_components = np.argmax(cumulative_variance >= 0.95) + 1  # 95% variance

        pca = PCA(n_components=pca_components)
        train_data_reduced = pca.fit_transform(train_data_balanced)
        test_data_reduced = pca.transform(test_data)

        # Convert back to DataFrame
        train_data_reduced = pd.DataFrame(train_data_reduced)
        test_data_reduced = pd.DataFrame(test_data_reduced)
    elif dim_reduction_method == 'correlation':
        # Adjust correlation threshold based on data
        corr_matrix = train_data_balanced.corr().abs()
        upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        high_corr_columns = [column for column in upper_triangle.columns if any(upper_triangle[column] > corr_threshold)]

        # Drop highly correlated columns
        train_data_reduced = train_data_balanced.drop(columns=high_corr_columns)
        test_data_reduced = test_data.drop(columns=high_corr_columns)
    else:
        train_data_reduced = train_data_balanced.copy()
        test_data_reduced = test_data.copy()

    # Ensure Consistent Columns #
    common_columns = train_data_reduced.columns.intersection(test_data_reduced.columns)
    train_data_final = train_data_reduced[common_columns]
    test_data_final = test_data_reduced[common_columns]

    # Add Target Column Back #
    
    # Reset indices of both data and target
    train_data_final = train_data_final.reset_index(drop=True)
    train_target_balanced = train_target_balanced.reset_index(drop=True)
    train_data_final['target'] = train_target_balanced

    test_data_final = test_data_final.reset_index(drop=True)
    test_target = test_target.reset_index(drop=True)
    test_data_final['target'] = test_target
    


    # Save the Preprocessed Data #
    # Create folder names based on methods used
    null_str = folder_path
    imbalance_str = imbalance_method if imbalance_method != 'none' else 'no_imbalance'
    dim_str = dim_reduction_method if dim_reduction_method != 'none' else 'no_reduction'

    folder_name = f"{null_str}_{imbalance_str}_{dim_str}"
    total_folder_list.append(folder_name)
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    train_filename = os.path.join(folder_name, "train_preprocessed.csv")
    test_filename = os.path.join(folder_name, "test_preprocessed.csv")

    train_data_final.to_csv(train_filename, index=False)
    test_data_final.to_csv(test_filename, index=False)

    print(f"Saved preprocessed data in folder: {folder_name}")
    return train_data_final, test_data_final



In [9]:

### Function to Generate all Possible Combinations ###
def generate_possible_combinations():
    null_methods = ['mean', 'median']
    null_thresholds = [10, 20]
    imbalance_methods = ['none', 'undersampling', 'smote', 'both']
    dim_reduction_methods = ['none', 'pca', 'correlation']

    possible_combinations = []
    for null_method in null_methods:
        for null_threshold in null_thresholds:
            for im_method in imbalance_methods:
                for dim_method in dim_reduction_methods:
                    if dim_method == 'pca':
                        # PCA with dynamic components
                        possible_combinations.append((null_method, null_threshold, im_method, dim_method, None))
                    else:
                        possible_combinations.append((null_method, null_threshold, im_method, dim_method, None))
    return possible_combinations



In [10]:
combinations = generate_possible_combinations()


In [11]:
total_folder_list = []

In [12]:


for comb in combinations:
    null_method, null_threshold, imbalance_method, dim_reduction_method, pca_components = comb

    preprocess_data(
        null_method=null_method,
        null_threshold=null_threshold,
        imbalance_method=imbalance_method,
        dim_reduction_method=dim_reduction_method,
        pca_components=pca_components,
        z_threshold=4 , # Z-score threshold for outlier handling
        total_folder_list = total_folder_list,
        corr_threshold = 0.8
    )


Imbalance Ratio: 12.929628133356418
Saved preprocessed data in folder: handeled_null_values\10_threshold\using_mean_no_imbalance_no_reduction
Imbalance Ratio: 12.929628133356418
Saved preprocessed data in folder: handeled_null_values\10_threshold\using_mean_no_imbalance_pca
Imbalance Ratio: 12.929628133356418
Saved preprocessed data in folder: handeled_null_values\10_threshold\using_mean_no_imbalance_correlation
Imbalance Ratio: 12.929628133356418
Saved preprocessed data in folder: handeled_null_values\10_threshold\using_mean_undersampling_no_reduction
Imbalance Ratio: 12.929628133356418
Saved preprocessed data in folder: handeled_null_values\10_threshold\using_mean_undersampling_pca
Imbalance Ratio: 12.929628133356418
Saved preprocessed data in folder: handeled_null_values\10_threshold\using_mean_undersampling_correlation
Imbalance Ratio: 12.929628133356418
Saved preprocessed data in folder: handeled_null_values\10_threshold\using_mean_smote_no_reduction
Imbalance Ratio: 12.9296281333

In [13]:
print(len(total_folder_list))

48


### **Scaling the data**

In [14]:
def apply_scaling(folder_path):
   
    train_file = os.path.join(folder_path, 'train_preprocessed.csv')
    test_file = os.path.join(folder_path, 'test_preprocessed.csv')
    
    train_data = pd.read_csv(train_file)
    test_data = pd.read_csv(test_file)
    
    train_target = train_data['target']
    test_target = test_data['target']
    train_features = train_data.drop(columns=['target'])
    test_features = test_data.drop(columns=['target'])
    
    scaler = StandardScaler()
    
    # Fit the scaler on training data, we will use it to transform the test data as well
    scaler.fit(train_features)
    
    # Transform both training and testing data
    train_features_scaled = pd.DataFrame(scaler.transform(train_features), columns=train_features.columns)
    test_features_scaled = pd.DataFrame(scaler.transform(test_features), columns=test_features.columns)
    
    # Add the target column back
    train_data_scaled = train_features_scaled.copy()
    train_data_scaled['target'] = train_target.reset_index(drop=True)
    test_data_scaled = test_features_scaled.copy()
    test_data_scaled['target'] = test_target.reset_index(drop=True)
    
    # Save the scaled datasets
    train_scaled_file = os.path.join(folder_path, 'train_scaled.csv')
    test_scaled_file = os.path.join(folder_path, 'test_scaled.csv')
    train_data_scaled.to_csv(train_scaled_file, index=False)
    test_data_scaled.to_csv(test_scaled_file, index=False)
    
    print(f"Scaling applied using StandardScaler and saved to: {train_scaled_file} and {test_scaled_file}")
    return None


In [15]:
for folder_path in total_folder_list:
    apply_scaling(folder_path)

Scaling applied using StandardScaler and saved to: handeled_null_values\10_threshold\using_mean_no_imbalance_no_reduction\train_scaled.csv and handeled_null_values\10_threshold\using_mean_no_imbalance_no_reduction\test_scaled.csv
Scaling applied using StandardScaler and saved to: handeled_null_values\10_threshold\using_mean_no_imbalance_pca\train_scaled.csv and handeled_null_values\10_threshold\using_mean_no_imbalance_pca\test_scaled.csv
Scaling applied using StandardScaler and saved to: handeled_null_values\10_threshold\using_mean_no_imbalance_correlation\train_scaled.csv and handeled_null_values\10_threshold\using_mean_no_imbalance_correlation\test_scaled.csv
Scaling applied using StandardScaler and saved to: handeled_null_values\10_threshold\using_mean_undersampling_no_reduction\train_scaled.csv and handeled_null_values\10_threshold\using_mean_undersampling_no_reduction\test_scaled.csv
Scaling applied using StandardScaler and saved to: handeled_null_values\10_threshold\using_mean_un

### **Save the folder path list in json file**

In [16]:
import json



file_name = "final_folders_list.json"

with open(file_name, 'w') as json_file:
    json.dump(total_folder_list, json_file)

print(f"The list has been saved to {file_name}")

The list has been saved to final_folders_list.json
